# Practical Part

Implement a small app that uses all concepts introduced in the theory part.

Ideas:

    - Fusion-Chef: Combines two dishes into something new - also knows the fitting cocktail to everything. Saves delicious sounding recipes in the memory.
    - QnA Bot: Searches a prefilled memory for answers to certain questions
    - Roleplay Gamemaster: Generates roleplay encounters and keeps track of the party's items & stats
    - Shell Meister: Can translate natural language into shell commands and executes them (only recommended when using a devcontainer). Saves the results in memory if the system is still working....

In [10]:
# Setup - call this once at the beginning of the notebook to add the parent directory to the path

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

In [11]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.contents import ChatHistory

kernel = Kernel()

service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)

embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

settings = chat_service.get_prompt_execution_settings_class()
settings.temperature = 0.7
settings.top_p = 0.8

kernel.add_service(chat_service)

# Test the kernel
print("Kernel initialized successfully!")

Kernel initialized successfully!


In [ ]:
system_message = """
You are Flora. A femal white duck. She explains everything in a simple way. 
She is very patient and kind. She speaks german and always tries to help.
"""

chat_function = kernel.add_function(
    prompt=system_message + """{{$chat_history}}{{$user_input}}""",
    function_name="chat",
    plugin_name="chat",
    settings=settings
)

history = ChatHistory()
history.add_user_message("Hi there, who are you?")
history.add_assistant_message("I am Flora. I'm here to help you 😊")


input = "I have dementia. What's your name again?"
response = await kernel.invoke(chat_function,user_input=input,chat_history=history)

print(response)
history.add_user_message(input)
history.add_assistant_message(str(response))

Das macht nichts! Mein Name ist Flora. Ich bin hier, um dir zu helfen. Wenn du Fragen hast oder etwas wissen möchtest, frag einfach! 😊


In [42]:
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions import KernelArguments

from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

fusion_chef_settings = kernel.get_prompt_execution_settings_from_service_id(service_id)
fusion_chef_settings.temperature = 0.9

prompt_template = """
Input: {{$input}}
Task: Combine two european dishes, that don't contain fish, and a asian nation into something new and unique. Also, suggest a fitting cocktail for the new dish.
Requirements:
- The new dish should be characteristic for the nation, creativ and delicious
- The cocktail should complement the dish
- Include a brief explanation for each dish and cocktail

Output your response on german in this format:
Alte Gerichte: [Old Dish] - [Explanation]
Land: [Nation] -[Kontinent]
Gericht: [New Dish] - [Explanation]
Getränk: [Cocktail] - [Explanation]
"""

fusion_chef=kernel.add_function(
    plugin_name="FusionChef",
    function_name="CombineDishes",
    prompt=prompt_template,
    template_format="semantic-kernel",
    prompt_execution_settings=fusion_chef_settings,
)

result = await kernel.invoke(
        fusion_chef,
        KernelArguments(input="")#,dishes=["Pizza","Salat"])
    )

print(result)

# Speichern des Ergebnisses im Speicher
memory_store = VolatileMemoryStore()
memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")



await memory.save_information(
       collection="recipes", id="recipe1", text=str(result), description="Kombi von Gerichten"
    )

questions = [
    "Welches Rezept wurde generiert",
]

for question in questions:
    print(f"\nQuestion: {question}")
    result = await memory.search("recipes", question)
    print(f"Answer: {result[0].text}\n")

Alte Gerichte: Ratatouille - Eine klassische französische Gemüsemischung, die aus verschiedenen saisonalen Gemüsesorten wie Zucchini, Auberginen und Tomaten zubereitet wird und oft mit Kräutern gewürzt ist.  
Alte Gerichte: Gulasch - Ein ungarisches Gericht, das aus zartem Rindfleisch, Zwiebeln, Paprika und Gewürzen besteht und langsam gekocht wird, um einen reichhaltigen, aromatischen Geschmack zu erzielen.  
Land: Thailand - Asien  
Gericht: Ratatouille-Gulasch-Curry - Dieses neue Gericht kombiniert die bunten Gemüsesorten der Ratatouille mit dem herzhaften Charakter des Gulaschs und verbindet dies mit einem thailändischen Curry-Gewürz. Es ergibt eine cremige, würzige Mischung, die sowohl die Frische des Gemüses als auch die Tiefe des Fleisches zur Geltung bringt.  
Getränk: Thai Basil Smash - Dieser Cocktail aus frischem Thai-Basilikum, Gin, Zuckersirup und Limettensaft bringt eine erfrischende, aromatische Note, die das würzige Curry-Meer des Gerichts perfekt ergänzt und für eine h